In [9]:
import numpy as np
import random

In [14]:
# foi utitlizado chat gpt como auxilio para entender o codigo e a implementação em si

# Ambiente do Mundo de Wumpus
TAMANHO = 4
CAVERNA = [
    [' ', ' ', ' ', ' '],
    ['W', ' ', 'A', ' '],
    [' ', ' ', 'O', 'A'],
    [' ', 'A', ' ', ' ']
]

# Recompensas
RECOMPENSAS = {
    'ouro': 100,
    'abismo': -100,
    'wumpus': -100,
    'andar': -1,
    'escalar_com_ouro': 50,
    'atirar': -10
}

ACTIONS = ['norte', 'sul', 'leste', 'oeste', 'pegar', 'atirar', 'escalar']

# Função para resetar o ambiente
def reset_env():
    return 0, 0, 1, 0, 1  # x, y, tem_flecha, tem_ouro, wumpus_vivo

# Função para executar uma ação
# Retorna novo estado, recompensa, done

def step(state, action):
    x, y, tem_flecha, tem_ouro, wumpus_vivo = state
    reward = RECOMPENSAS['andar']
    done = False
    caverna = [row[:] for row in CAVERNA]

    if action == 0:  # norte
        if x > 0:
            x -= 1
    elif action == 1:  # sul
        if x < TAMANHO-1:
            x += 1
    elif action == 2:  # leste
        if y < TAMANHO-1:
            y += 1
    elif action == 3:  # oeste
        if y > 0:
            y -= 1
    elif action == 4:  # pegar
        if caverna[x][y] == 'O':
            tem_ouro = 1
            reward = RECOMPENSAS['ouro']
            caverna[x][y] = ' '
    elif action == 5:  # atirar
        if tem_flecha:
            tem_flecha = 0
            reward = RECOMPENSAS['atirar']
            # Simples: se Wumpus está na linha ou coluna, mata
            if wumpus_vivo and (x == 1 and y <= 1):
                wumpus_vivo = 0
    elif action == 6:  # escalar
        if x == 0 and y == 0:
            if tem_ouro:
                reward = RECOMPENSAS['escalar_com_ouro']
                done = True
            else:
                reward = 0
                done = True

    # Verifica se caiu em abismo
    if caverna[x][y] == 'A':
        reward = RECOMPENSAS['abismo']
        done = True
    # Verifica se encontrou Wumpus
    if caverna[x][y] == 'W' and wumpus_vivo:
        reward = RECOMPENSAS['wumpus']
        done = True
    # Se pegou ouro, não termina o jogo, só se escalar
    return (x, y, tem_flecha, tem_ouro, wumpus_vivo), reward, done

# Q-learning simples
def train_qlearning(episodios=500):
    Q = {}
    alpha = 0.1
    gamma = 0.9
    epsilon = 0.2
    for ep in range(episodios):
        state = reset_env()
        done = False
        total_reward = 0
        for t in range(100):
            s_key = tuple(state)
            if s_key not in Q:
                Q[s_key] = np.zeros(len(ACTIONS))
            # Política epsilon-greedy
            if random.random() < epsilon:
                action = random.randint(0, len(ACTIONS)-1)
            else:
                action = np.argmax(Q[s_key])
            next_state, reward, done = step(state, action)
            ns_key = tuple(next_state)
            if ns_key not in Q:
                Q[ns_key] = np.zeros(len(ACTIONS))
            Q[s_key][action] += alpha * (reward + gamma * np.max(Q[ns_key]) - Q[s_key][action])
            state = next_state
            total_reward += reward
            if done:
                break
        if (ep+1) % 50 == 0:
            print(f"Episódio {ep+1}: recompensa total = {total_reward}")
    return Q

if __name__ == "__main__":
    Q = train_qlearning(300)
    print("\nTreinamento finalizado!")
    # Teste: executar uma trajetória greedy
    state = reset_env()
    done = False
    print("\nTrajetória greedy:")
    for _ in range(30):
        print(f"Posição: ({state[0]+1},{state[1]+1}), Ouro: {state[3]}, Wumpus vivo: {state[4]}")
        s_key = tuple(state)
        if s_key in Q:
            action = np.argmax(Q[s_key])
        else:
            action = random.randint(0, len(ACTIONS)-1)
        state, reward, done = step(state, action)
        if done:
            print("Fim do episódio.")
            break

Episódio 50: recompensa total = 0
Episódio 100: recompensa total = -118
Episódio 150: recompensa total = 0
Episódio 200: recompensa total = 0
Episódio 250: recompensa total = 0
Episódio 300: recompensa total = 0

Treinamento finalizado!

Trajetória greedy:
Posição: (1,1), Ouro: 0, Wumpus vivo: 1
Fim do episódio.
